In [1]:
from archappl.client import ArchiverDataClient
from phantasy import MachinePortal
from datetime import datetime

In [2]:
mp = MachinePortal("FRIB", "MEBT2FS1A")
bpms = mp.get_elements(type="BPM")
client = ArchiverDataClient()
client.url = "http://epicsarchiver0.ftc:17668"

INFO: 2019-06-07 12:10:29,551: phantasy.library.parser.config: Searching configuration in relative or absolute path: 'FRIB'
INFO: 2019-06-07 12:10:29,551: phantasy.library.parser.config: Searching configuration under path: '/usr/lib/phantasy-machines' 'FRIB'
INFO: 2019-06-07 12:10:29,552: phantasy.library.parser.config: Importing 'FRIB' from '/usr/lib/phantasy-machines/FRIB'
INFO: 2019-06-07 12:10:29,554: phantasy.library.parser.config: Using config file: phantasy.ini
INFO: 2019-06-07 12:10:29,627: phantasy.library.operation.lattice: Using CSV/SQLite instead of CFS '/usr/lib/phantasy-machines/FRIB/./mebt2fs1a/channels.csv'
INFO: 2019-06-07 12:10:29,628: phantasy.library.pv.datasource: Set source to be CSV table: channels.csv.
INFO: 2019-06-07 12:10:29,709: phantasy.library.operation.lattice: Model data directory: /user/zhangt/phantasy_data/20190607/model_data/data_4n4r__ut
INFO: 2019-06-07 12:10:29,717: phantasy.library.operation.lattice: Found 2161 PVs in MEBT2FS1A.
INFO: 2019-06-07 1

In [3]:
pvs_x = [i.pv('X', handle='readback')[0] for i in bpms]
pvs_y = [i.pv('Y', handle='readback')[0] for i in bpms]

In [4]:
ts0 = "2019-03-05T13:04:08.120000"
dt = datetime.strptime(ts0, "%Y-%m-%dT%H:%M:%S.%f")

In [5]:
ts0_epoch = dt.timestamp()

In [6]:
_ts0 = datetime.fromtimestamp(ts0_epoch).isoformat() + "-05:00"

In [7]:
print(_ts0)

2019-03-05T13:04:08.120000-05:00


In [8]:
r = client.get_data_at_time([pvs_x[0]], ts=_ts0)

In [9]:
r

{'FE_MEBT:BPM_D1056:XPOS_RD': {'nanos': 4464964,
  'secs': 1551809048,
  'severity': 0,
  'status': 0,
  'val': -0.944449345999152}}

In [10]:
client

[Data Client] Archiver Appliance on: http://epicsarchiver0.ftc:17668/retrieval/data/getData.json

In [11]:
from archappl.client import ArchiverMgmtClient

In [12]:
admin_client = ArchiverMgmtClient()
admin_client.url = "http://epicsarchiver0.ftc:17665"

In [13]:
admin_client

[Admin Client] Archiver Appliance on: http://epicsarchiver0.ftc:17665/mgmt/bpl

# range

In [14]:
def iso_to_epoch(s,):
    # return datetime.timestamp, and string of timezone (e.g.-05:00)
    ts, tz = s[:-6], s[-6:]
    dt = datetime.strptime(ts, "%Y-%m-%dT%H:%M:%S.%f")
    return dt.timestamp(), tz

In [15]:
def epoch_to_iso(ts, tz='-05:00'):
    return datetime.fromtimestamp(ts).isoformat() + tz

In [16]:
ts0_iso = "2019-03-05T13:04:08.120000-05:00"
ts0_epoch, tz = iso_to_epoch(ts0_iso)

In [17]:
ts0_epoch

1551809048.12

In [18]:
delt_sec = 1000
t0 = ts0_epoch - delt_sec
t1 = ts0_epoch + delt_sec
ts_begin = epoch_to_iso(t0)
ts_end = epoch_to_iso(t1)
print(ts_begin, ts_end)

2019-03-05T12:47:28.120000-05:00 2019-03-05T13:20:48.120000-05:00


In [19]:
import numpy as np

In [79]:
pv_list = pvs_x[0::1]
n_ts = 200
n_elem = len(pv_list)
ts_list = [epoch_to_iso(i, tz) for i in np.linspace(t0, t1, n_ts)]
m = []
#m = np.zeros([n_ts, n_elem])

In [80]:
for i, its in enumerate(ts_list):
    r = client.get_data_at_time(pv_list, ts=its)
    m.append([r[pv]['val'] for pv in pv_list if pv in r])

In [81]:
results = np.array(m)

In [82]:
valid_pvs = [k for k in pv_list if k in list(r.keys())]

In [83]:
import pickle

In [84]:
data = {'results': results, 'pvs': valid_pvs, 'ts_iso': ts_list, 'ts': np.linspace(t0, t1, n_ts)}

In [85]:
with open('data.pkl', 'wb') as fp:
    pickle.dump(data, fp)